In [12]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
from typing import Dict
import sendgrid
import os
import asyncio
from pydantic import BaseModel
load_dotenv(override=True)

True

In [6]:

class ResultadoTeste(BaseModel):
    data: str
    vazao: float
    latencia: float
    erro: float

class Avaliacao(BaseModel):
    avaliacao: str
    baseline: ResultadoTeste
    resultado_bateria: ResultadoTeste


In [4]:
#Tolls Agent - Capturar a Baseline para parâmetros de desempenho
# Baseline - Entres as baterias de teste, representar a melhor performance

with open("recursos/baterias-de-teste.csv", "r", encoding="utf-8") as f:
    bateriasDeTeste = f.read()

instrucoes_baseline = """
    Você é um especialista em análise de desempenho de software.
    Sua tarefa é analisar as baterias de teste e identificar a melhor performance.
    deve definir como baseline o resultado final de uma das baterias de testes de desempenho já realizadas. 
    Baseline é uma bateria que obteve o melhor resultado, seguindo os seguintes critérios: latência mais baixa, 
    taxa de erro mais baixo e vazão mais alta. Você deve apontar com base nesta relação de csv de baterias de testes, 
    com data da execução, vazão, latência e Taxa de Erro, 
    Você não precisa ser detalhista, apenas apontar a melhor bateria de teste de desempenho.
    Deve-se retorna a melhor bateria nos critérios apontado acima, apontando no seguinte formato json: 
    {
        "data": "data de execução da bateria", 
        "vazao": "vazão alcançada na bateria", 
        "latencia": "A latência alcançada na bateria", 
        "erro": "taxa de erro alcançado na bateria"
    }
"""

baseline_agent = Agent(
        name="Seletor de Baseline",
        instructions=instrucoes_baseline,
        output_type=ResultadoTeste,
        model="gpt-5-mini",
)

baseline_tool = baseline_agent.as_tool(tool_name="baseline_agent", tool_description="Selecionar a melhor bateria de teste de desempenho")
baseline_tool


FunctionTool(name='baseline_agent', description='Selecionar a melhor bateria de teste de desempenho', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'baseline_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C16824C900>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [ ]:
#Tolls Agent - Capturar o melhor resultado alcançado em uma bateria de teste

with open("recursos/resultado-bateria1.csv", "r", encoding="utf-8") as f:
    bateriaEmanalise = f.read()

instrucoes_resultado_bateria = """
    Você é um especialista em análise de desempenho de software.
    Sua tarefa é analisar o resultado de uma bateria de teste e identificar o melhor resultado alcançado.
    O melhor resultado deve obedecer aos seguintes critérios: 
        latência não deve ser maior que 400ms, 
        taxa de erro não deve ser maior que 0.3% e 
        vazão deve ser a maior possível. 
    Você deve analisar os dados da bateria de teste no aquivo csv fornecido no com as seguintes colunas, 
    com data da execução, vazão, latência e Taxa de Erro, 
    Você não precisa ser detalhista, apenas apontar a melhor resultado alcançado.
    Deve-se retorna a melhor bateria nos critérios apontado acima, apontando no seguinte formato json: 
    {
        "data": "Deve ser a data atual", 
        "vazao": "vazão alcançada na bateria", 
        "latencia": "A latência alcançada na bateria", 
        "erro": "taxa de erro alcançado na bateria"
    }
"""
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
google_api_key = os.getenv('GOOGLE_API_KEY')
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.5-flash", openai_client=gemini_client)

resultado_bateria_agent = Agent(
        name="Capturar resultado",
        instructions=instrucoes_resultado_bateria,
        output_type=ResultadoTeste,
        model=gemini_model,
)

resultado_bateria_tool = resultado_bateria_agent.as_tool(tool_name="resultado_bateria_agent", tool_description="Selecionar a melhor bateria de teste de desempenho")
resultado_bateria_tool



FunctionTool(name='resultado_bateria_agent', description='Selecionar a melhor bateria de teste de desempenho', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'resultado_bateria_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C16873A2A0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [14]:
#Handoff Agent - Avaliar se o resultado da bateria de teste é melhor que a baseline
instrucoes_comparar = """
    Você é um especialista em análise de desempenho de software.
    Sua tarefa é comparar o resultado da bateria de teste com a baseline.
    você receberá do baseline_agent a baseline com os dados de data,vazao,latencia e erro.
    você receberá do resultado_bateria_agent o resultado da bateria de teste com os dados de data,vazao,latencia e erro.
    você deve avaliar se o resultado da bateria de teste é melhor que a baseline, seguindo os seguintes critérios: latência mais baixa, 
    taxa de erro mais baixo e vazão mais alta.
    Deve-se retorna a melhor bateria nos critérios apontado acima, apontando no seguinte formato json: 
    {
        "avaliacao": "Avaliação se o resultado da bateria de teste é melhor que a baseline",
        "baseline": {
            "data": "data de execução da bateria", 
            "vazao": "vazão alcançada na bateria", 
            "latencia": "A latência alcançada na bateria", 
            "erro": "taxa de erro alcançado na bateria"
        },
        "resultado_bateria": {
            "data": "data de execução da bateria", 
            "vazao": "vazão alcançada na bateria", 
            "latencia": "A latência alcançada na bateria", 
            "erro": "taxa de erro alcançado na bateria"
        }
    }
"""

comparar_agent = Agent(
    name="Comparar resultado",
    instructions=instrucoes_comparar,
    model="gpt-5-mini",
    output_type=Avaliacao,
    handoff_description="Comparar resultado da bateria de teste com a baseline")


In [17]:
#Orchestrator Agent - Orquestrador para comparar o resultado da bateria de teste com as baterias já realizadas
tools = [baseline_tool, resultado_bateria_tool]
handoff_tools = [comparar_agent]


instrucoes_orchestrator = """
    Você é um orquestrador para realizar o trabalho de comparar o resultado da bateria de teste com as baterias já realizadas.
    você deverá seguir os seguintes passos:
    1 - Repassar a bateriaEmanalise para o resultado_bateria_agent
    2 - Receber o resultado da bateria de teste do resultado_bateria_agent
    3 - Receber a baseline da baseline_agent
    4 - Repassar a baseline para o comparar_agent
    5 - realizar o handoff para o comparar_agent para comparar o resultado da bateria de teste com a baseline
    6 - Retornar o resultado da comparação para o resultado_bateria_agent
"""

orchestrator = Agent(
    name="Orchestrator",
    instructions=instrucoes_orchestrator,
    tools=tools,
    handoffs=handoff_tools,
    model="gpt-5-mini")

message = f"""
    realize a comparação do resultado da bateria de teste com a baseline
    a bateria de teste é: {bateriaEmanalise}
    a baseline deve ser selecionada entre as baterias: {bateriasDeTeste}
"""


with trace("Analisar desempenho"):
    result = await Runner.run(orchestrator, message)

print(result.final_output)

avaliacao='O resultado da bateria de teste é pior que a baseline: vazão inferior (195.8 < 410), latência maior (320 > 300) e taxa de erro maior (0.3 > 0.0).' baseline=ResultadoTeste(data='2025-11-14', vazao=410.0, latencia=300.0, erro=0.0) resultado_bateria=ResultadoTeste(data='2023-10-27', vazao=195.8, latencia=320.0, erro=0.3)
